# Topic Modelling Tutorial

Topic modelling is a widely-used method of exploring the semantic and discursive concepts, or "topics", within collections of texts. Wikipedia defines a topic model as follows:

> In machine learning and natural language processing, a topic model is a type of statistical model for discovering the abstract "topics" that occur in a collection of documents. Topic modeling is a frequently used text-mining tool for discovery of hidden semantic structures in a text body. Intuitively, given that a document is about a particular topic, one would expect particular words to appear in the document more or less frequently: "dog" and "bone" will appear more often in documents about dogs, "cat" and "meow" will appear in documents about cats, and "the" and "is" will appear approximately equally in both. A document typically concerns multiple topics in different proportions; thus, in a document that is 10% about cats and 90% about dogs, there would probably be about 9 times more dog words than cat words. The "topics" produced by topic modeling techniques are clusters of similar words. A topic model captures this intuition in a mathematical framework, which allows examining a set of documents and discovering, based on the statistics of the words in each, what the topics might be and what each document's balance of topics is.

If you are interested in getting a better understanding of what topic modelling does and how it works, two quick reads are Matt Jockers' <a href="https://www.matthewjockers.net/macroanalysisbook/lda/" target="_blank">The LDA Buffet: A Topic Modeling Fable</a> and Ted Underwood's <a href="https://tedunderwood.com/2012/04/07/topic-modeling-made-just-simple-enough/" target="_blank">Topic Modeling Made Just Simple Enough</a>.

<a href="https://mimno.github.io/Mallet/" target="_blank">MALLET</a> is the most widely used topic modelling tool in the Digital Humanities, both because it is very performant and because its implementation of the Latent Dirichlet Allocation (LDA) algorithm tends to produce quality topics. MALLET is a command-line tool written in Java. It is independent of Lexos and must be installed separately. User-friendly instructions for installing and using MALLET can be found in the Programming Historian tutorial <a href="https://programminghistorian.org/en/lessons/topic-modeling-and-mallet" target="_blank">Getting Started with Topic Modeling and MALLET</a>.

One of the difficulties of using MALLET is that its output is relatively difficult to manipulate into data structures useful for visualisation. This tutorial demonstrates how to create a visualisation of a MALLET topic model using Andrew Goldstone's <a href="https://agoldst.github.io/dfr-browser/" target="_blank">Getting Started with Topic Modeling and MALLET</a>. If you want to see it in action, click the link and then follow the links to the demo, which is based on a model of a century of articles in the _Proceedings of the Modern Language Association (PMLA)_.

## Before Getting Started

Before you get started, make sure that you have a working installation of MALLET by following the instructions in the Programming Historian tutorial Getting Started with Topic Modeling and MALLET. Make sure that you know the path to the MALLET binary file.

## Import Some Data

For this tutorial, we'll use the English-language sample data (a selection of Wikipeia articles) that comes with MALLET. This a very small dataset and should run very quickly. In the cell below, change the `data_path` value to wherever your data is located.

In this tutorial, we will save our model and dfr-browser in a folder called `topic_model`. You can configure the path to that model in the cell below.

In [ ]:
# Configuration
data_path = "C:/mallet/mallet-2.0.8/sample-data/web/en"
model_dir = "topic_model"

# Python and Lexos imports
import os
from lexos.io.smart import Loader

# Make the model directory if it doesn't exist
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

# Load the data
loader = Loader()
loader.load(data_path)
print("Extract of first text:\n")
print(loader.texts[0][0:1245])

## Create Metadata

Although not required for topic modelling, metadata is needed to generate a dfr-browser. Since the MALLET sample data does not come with metadata, we will quickly generate some arbitrary metadata based on the Wikipedia articles for demonstration purposes.

Dfr-browser was originally designed for displaying models of journal articles in the JSTOR database, so you need to supply metadata fields with the categries it expects. These categories are `id`, `title`, `publication`, `authors`, `volume`, `issue`, `year`, and `pagerange`. If these categories are not appropriate to your data, you can leave them blank (as an empty string). You can also include additional fields (we add `file` and `url` below), although they may not be displayed in the dfr-browser.

The easiest way to do this is to create a CSV file called `meta.csv` with no headings and store it in your `topic_model` directory. If you have your own metadata CSV file, skip the next cell.

In [ ]:
import pandas as pd

metadata = [
  {"id": "1", "title": "Elizabeth Needham", "authors": "Fearne McKnight", "publication": "Wikipedia","volume":"","issue":"","year":"1731","pagerange":"","file":"elizabeth_needham.txt","url": "https://en.wikipedia.org/wiki/Elizabeth_Needham"},
  {"id": "2", "title": "Equipartition Theorem", "authors": "Jayson Clayton", "publication": "Wikipedia","volume":"","issue":"","year":"1843","pagerange":"","file":"equipartition_theorem.txt.txt","url": "https://en.wikipedia.org/wiki/Equipartition_theorem"},
  {"id": "3", "title": "Gunnhild konungamóðir", "authors": "Tyrone Bennett", "publication": "Wikipedia","volume":"","issue":"","year":"980","pagerange":"","file":"gunnild.txt","url": "https://en.wikipedia.org/wiki/Gunnhild_Mother_of_Kings"},
  {"id": "4", "title": "Richard Hawes", "authors": "Nichola Cantrell","publication": "Wikipedia","volume":"","issue":"","year":"1877","pagerange":"","file":"hawes.txt","url": "https://en.wikipedia.org/wiki/Richard_Hawes"},
  {"id": "5", "title": "Clem Hill", "authors": "Dolores Mitchell", "publication": "Wikipedia", "volume": "","issue":"","year":"1945","pagerange":"","file":"hill.txt","url": "https://en.wikipedia.org/wiki/Clem_Hill"},
  {"id": "6", "title": "The Battle of Shiloh", "authors": "Johnny Buxton", "publication": "Wikipedia","volume":"","issue":"","year":"1862","pagerange":"","file":"shiloh.txt","url": "https://en.wikipedia.org/wiki/Battle_of_Shiloh"},
  {"id": "7", "title": "Sunderland Echo", "authors": "Sulaiman Connor", "publication": "Wikipedia","volume":"","issue":"","year":"1873","pagerange":"","file":"sunderland_echo.txt","url": "https://en.wikipedia.org/wiki/Sunderland_Echo"},
  {"id": "8", "title": "Thespis", "authors": "Miguel John","publication": "Wikipedia","volume":"","issue":"","year":"1871","pagerange":"","file":"thespis.txt","url": "https://en.wikipedia.org/wiki/Thespis_(opera)"},
  {"id": "9", "title": "Thylacine", "authors": "Emmie Strong","publication": "Wikipedia","volume":"","issue":"","year":"1930","pagerange":"","file":"thylacine.txt","url": "https://en.wikipedia.org/wiki/Thylacine"},
  {"id": "10", "title": "The Rings of Uranus", "authors": "Olivia-Grace Mayo","publication": "Wikipedia","volume":"","issue":"","year":"1977","pagerange":"","file":"uranus.txt","url": "https://en.wikipedia.org/wiki/Uranus"},
  {"id": "11", "title": "Robert Sterling Yard", "authors": "Viola Meyers","publication": "Wikipedia","volume":"","issue":"","year":"1945","pagerange":"","file":"yard.txt","url": "https://en.wikipedia.org/wiki/Robert_Sterling_Yard"},
  {"id": "12", "title": "Preity Zinta", "authors": "Felicity Deleon","publication": "Wikipedia","volume":"","issue":"","year":"1975","pagerange":"","file":"zinta.txt","url": "https://en.wikipedia.org/wiki/Preity_Zinta"}
]
metadata_file = f"{model_dir}/meta.csv"
df = pd.DataFrame(metadata)
# df.to_csv(metadata_file, index=False, header=False)
df

If you have your own `meta.csv` file, you can run the next cell to inspect contents. If you generated test metadata in the cell above, this cell should have the same output.

In [ ]:
metadata_file = f"{model_dir}/meta.csv"
df = pd.read_csv(metadata_file)
df

## Scrub the Data

Now we will use Lexos to scrub the data. We import the `Scrubber` components, make a pipeline, and run the pipeline on each text. The components here are just random samples of the possible options.

In [ ]:
from lexos.scrubber.pipeline import make_pipeline, pipe
from lexos.scrubber.registry import scrubber_components, load_components

emails, new_lines, pattern = load_components(("emails", "new_lines", "pattern"))
scrub = make_pipeline(
    emails,
    new_lines,
    pipe(pattern, pattern="\'")
)
data = [scrub(item) for item in loader.texts]

### Tokenise the Data

We will import the Lexos tokenizer and create a list of spaCy docs. In the example below, we use spaCy's "en_core_web_sm" language model, and we'll add "gorillas" as an arbitrary extra stop word.

Keep in mind that each token in the doc is annotated with its part of speech, whether or not it is a stop word, and whether or not it is a punctuation mark (to name a few examples). We will use these properties below.

Note that because tokenisation also involves adding these annotations, it may take a long time for large datasets.

In [ ]:
from lexos import tokenizer

docs = tokenizer.make_docs(
    data,
    model="en_core_web_sm",
    add_stopwords=["gorillas"]
)

## Topic Modelling¤

We are now ready to create the topic model. We start by creating a `Mallet` object, pointing it to a directory where we would like the model to be saved, and supplying the path to our MALLET installation (you may need to change the `mallet_path` value below).

In [ ]:
from lexos.topic_model.mallet import Mallet

model = Mallet(
    model_dir=model_dir,
    mallet_path="C:/mallet/mallet-2.0.8/bin"
)

### Import the Data into MALLET

We use our `Mallet` object to import our tokenised docs. In the example below, we will import only tokens labelled as nouns. The default behaviour is to skip stop words and punctuation.

This process creates two files in the model directory. The first is called `data.txt`. This file contains all our doc tokens with one doc per line. Each doc is a bag of words (meaning token order is lost). The second file is called `import.mallet`. This contains the information in `data.txt`, imported into a binary format.

In [ ]:
model.import_data(docs, allowed=["NOUN"])

## Train the Model

If we've followed the procedure above, we can simply call `model.train()`. If for some reason, we need to re-instantiate the `Mallet` object, we can do so and skip the import step above. In this case, we would call `model.train(mallet_file="import.mallet")`.

**Note:** The progress of the modelling task is monitored by continuous output to the console, or, in this case, the Jupyter notebook. The `%%capture` line at the beginning of the cell prevents the output from being printed and thus overwhelming the memory buffer.

In [ ]:
%%capture

model.train()

Once the model is complete (which may take a long time if you have a lot of data but should take seconds for the MALLET sample data), it is worth inspecting the model. Navigate to your model's `keys.txt` file and open it. If some topics have no keywords, that is a sign that something has gone wrong with your model. If everything looks good, you're ready for the next step.

### Create the Topic Scale File

dfr-browser requires an additional CSV file containing topic scaling data. To produce this, we just need to call `model.scale()`.

In [ ]:
model.scale()

## Build Dfr-Browser

We can now generate a dfr-browser from our topic model. We import the `DfrBrowser` class and create a `DfrBrowser` object. This will create a `dfr_browser` directory in your model's folder where all the necessary files are housed. Make sure that your `meta.csv` file is in the root of your topic model folder.

In [ ]:
from lexos.topic_model.dfr_browser import DfrBrowser

browser = DfrBrowser(model_dir=model_dir)

docs_dir = f"{model_dir}/dfr_browser/data/docs"
print(f"Copying text data to {docs_dir}...")
if not os.path.exists(docs_dir):
    os.mkdir(docs_dir)
for i, text in enumerate(loader.texts):
    with open(f"{docs_dir}/{i}.txt", "w", encoding="utf-8") as f:
        f.write(text)
print("Done.")

### Open Dfr-Browser

When the process is complete, you will need to start a local server. Open a command prompt and `cd` to your model's `dfr_browser` folder. Then type `python -m http.server 8080`. If you are already running a local server on port 8080, you can change it to something else. Then point your browser to <a href="http://localhost:8080/" target="_blank">http://localhost:8080/</a>, and the dfr-browser should load.

Note that some features of dfr-browser may not work if you do not have appropriate metadata.

When you are finished, remember to go back to the command prompt and type `Ctr+C` to shutdown the server.

## Using Your Own Data

Topic modelling works best with large datasets. However, getting such data can be a pain. A good place to start is Alan Liu's <a href="http://dhresourcesforprojectbuilding.pbworks.com/w/page/69244469/Data%20Collections%20and%20Datasets#demo-corpora" target="_blank">cultivated list of corpora</a>. Keep in mind that you may need to do some work to ensure that you have appropriate metadata, and you may have to massage it into the format required by dfr-browser.

Your ultimate goal is to download (or identify urls for) your texts so that they can be loaded using the Lexos `Loader` and to create a `meta.csv` file in your project's `topic_model` folder. Then you can jump into the procedure above, starting with the scrubbing and tokenising cells. Keep in mind that large datasets will take a very long time to tokenise if you are not using the default model. Make sure to disable any pipeline components you are not going to use (see the Tokenizer Tutorial for information on disabling components). 